## Part 3: Modelling & Predicting Pneumonia w/ Neural Networks

In [1]:
# Imports
import os
import cv2
import glob
import time
import pydicom
import skimage
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import feature, filters
%matplotlib inline

from functools import partial
from collections import defaultdict
from joblib import Parallel, delayed
from lightgbm import LGBMClassifier
from tqdm import tqdm

# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16
from keras import models
from keras import layers

# sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

sns.set_style('whitegrid')
np.warnings.filterwarnings('ignore')

In [2]:
# List our paths
trainImagesPath = "../input/rsna-pneumonia-detection-challenge/stage_2_train_images"
testImagesPath = "../input/rsna-pneumonia-detection-challenge/stage_2_test_images"

labelsPath = "../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
classInfoPath = "../input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv"

# Read the labels and classinfo
labels = pd.read_csv(labelsPath)
details = pd.read_csv(classInfoPath)

## Part 3.1: Attaining our Training & Testing Data in Proper Format

In [3]:
"""
@Description: Reads an array of dicom image paths, and returns an array of the images after they have been read

@Inputs: An array of filepaths for the images

@Output: Returns an array of the images after they have been read
"""
def readDicomData(data):
    
    res = []
    
    for filePath in tqdm(data): # Loop over data
        
        # We use stop_before_pixels to avoid reading the image (Saves on speed/memory)
        f = pydicom.read_file(filePath, stop_before_pixels=True)
        res.append(f)
    
    return res

In [4]:
# Get an array of the test & training file paths
trainFilepaths = glob.glob(f"{trainImagesPath}/*.dcm")
testFilepaths = glob.glob(f"{testImagesPath}/*.dcm")

# Read data into an array
trainImages = readDicomData(trainFilepaths[:5000])
testImages = readDicomData(testFilepaths)

## Part 3.2: Balancing our Data

We balance our data as CNNs work best on evenly balanced data

In [5]:
COUNT_NORMAL = len(labels.loc[labels['Target'] == 0]) # Number of patients with no pneumonia
COUNT_PNE = len(labels.loc[labels['Target'] == 1]) # Number of patients with pneumonia
TRAIN_IMG_COUNT = len(trainFilepaths) # Total patients

# We calculate the weight of each
weight_for_0 = (1 / COUNT_NORMAL)*(TRAIN_IMG_COUNT)/2.0 
weight_for_1 = (1 / COUNT_PNE)*(TRAIN_IMG_COUNT)/2.0

classWeight = {0: weight_for_0, 
               1: weight_for_1}

print(f"Weights: {classWeight}")

## Part 3.3: Get Train_Y & Test_Y

In [6]:
"""
@Description: This function parses the medical images meta-data contained

@Inputs: Takes in the dicom image after it has been read

@Output: Returns the unpacked data and the group elements keywords
"""
def parseMetadata(dcm):
    
    unpackedData = {}
    groupElemToKeywords = {}
    
    for d in dcm: # Iterate here to force conversion from lazy RawDataElement to DataElement
        pass
    
    # Un-pack Data
    for tag, elem in dcm.items():
        tagGroup = tag.group
        tagElem = tag.elem
        keyword = elem.keyword
        groupElemToKeywords[(tagGroup, tagElem)] = keyword
        value = elem.value
        unpackedData[keyword] = value
        
    return unpackedData, groupElemToKeywords

In [7]:
# These parse the metadata into dictionaries
trainMetaDicts, trainKeyword = zip(*[parseMetadata(x) for x in tqdm(trainImages)])
testMetaDicts, testKeyword = zip(*[parseMetadata(x) for x in tqdm(testImages)])

In [8]:
"""
@Description: This function goes through the dicom image information and returns 1 or 0
              depending on whether the image contains Pneumonia or not

@Inputs: A dataframe containing the metadata

@Output: Returns the Y result (i.e: our train and test y)
"""
def createY(df):
    y = (df['SeriesDescription'] == 'view: PA')
    Y = np.zeros(len(y)) # Initialise Y
    
    for i in range(len(y)):
        if(y[i] == True):
            Y[i] = 1
    
    return Y


train_df = pd.DataFrame.from_dict(data=trainMetaDicts)
test_df = pd.DataFrame.from_dict(data=testMetaDicts)

train_df['dataset'] = 'train'
test_df['dataset'] = 'test'

df = train_df
df2 = test_df

train_Y = createY(df) # Create training Y 
test_Y = createY(df2) # Create testing Y

## Part 3.4: Get Train_X & Test_X

In [9]:
"""
@Description: This decodes an image by reading the pixel array, resizing it into the correct format and
              normalising the pixels

@Inputs:
    - filePath: This is the filepath of the image that we want to decode

@Output:
    - img: This is the image after it has been decoded
"""
def decodeImage(filePath):
    image = pydicom.read_file(filePath).pixel_array
    image = cv2.resize(image, (128, 128))
    return (image/255)

In [10]:
# Get our train x in the correct shape
train_X = []

for filePath in tqdm(trainFilepaths[:5000]):
    
    img = decodeImage(filePath)
    train_X.append(img)

train_X = np.array(train_X) # Convert to np.array
train_X_rgb = np.repeat(train_X[..., np.newaxis], 3, -1) # Reshape into rgb format

In [11]:
# Get our test x in the correct shape for NN
test_X = []

for filePath in tqdm(testFilepaths):
    img_test = decodeImage(filePath) # Decode & Resize
    test_X.append(img_test)

test_X = np.array(test_X) # Convert to np array
test_X_rgb = np.repeat(test_X[..., np.newaxis], 3, -1) # Reshape into rgb format

In [12]:
"""
@Description: This function plots our metrics for our models across epochs

@Inputs: The history of the fitted model

@Output: N/A
"""
def plottingScores(hist):
    fig, ax = plt.subplots(1, 5, figsize=(20, 3))
    ax = ax.ravel()

    for i, met in enumerate(['accuracy', 'precision', 'recall', 'AUC', 'loss']):
        ax[i].plot(hist.history[met])
        ax[i].plot(hist.history['val_' + met])
        ax[i].set_title('Model {}'.format(met))
        ax[i].set_xlabel('epochs')
        ax[i].set_ylabel(met)
        ax[i].legend(['train', 'val'])

## Part 3.5: Metrics Evaluation
For our metrics, we want to include <b><i>precision</i></b> and <b><i>recall</i></b> as they will provide use with more info on how good our model is
 
 
- <b><u>Accuracy:</u></b> This tells us what fraction of the labels are correct.
    - Since our data is not balanced, accuracy might give a skewed sense of a good model


- <b><u>Precision:</u></b> This tells us the number of true positives (TP) over the sum of TP and false positives (FP). 
    - It shows what fraction of labeled positives are actually correct.


- <b><u>Recall:</u></b> The number of TP over the sum of TP and false negatves (FN). 
    - It shows what fraction of actual positives are correct.

In [13]:
# These our our scoring metrics that are going to be used to evaluate our models
METRICS = ['accuracy', 
           tf.keras.metrics.Precision(name='precision'), 
           tf.keras.metrics.Recall(name='recall'), 
           tf.keras.metrics.AUC(name='AUC')]

### Tuning our Models with Callbacks

- We'll use Keras callbacks to further finetune our model. 
- The <b>checkpoint callback</b> saves the best weights of the model, so next time we want to use the model, we do not have to spend time training it. 
- The <b>early stopping callback</b> stops the training process when the model starts becoming stagnant, or even worse, when the model starts overfitting. 
- Since we set restore_best_weights to True, the returned model at the end of the training process will be the model with the best weights (i.e. low loss and high accuracy).

In [14]:
# Define our callback functions to pass when fitting our NNs
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

## Part 3.6: Building Model #1 - Fully Connected Model

In [32]:
"""
@Description: This function builds our simple Fully-connected NN

@Inputs: N/A

@Output: Returns the FCNN Model
"""
def build_fcnn_model():
    
    # Basic model with a flattening layer followng by 2 dense layers
    # The first dense layer is using relu and the 2nd one is using sigmoid
    model = tf.keras.models.Sequential([
                tf.keras.layers.Flatten(input_shape = (128, 128, 3)), 
                tf.keras.layers.Dense(128, activation = "relu"),
                tf.keras.layers.Dense(64, activation = "relu"),
                tf.keras.layers.Dense(32, activation = "relu"), 
                tf.keras.layers.Dense(1, activation = "sigmoid")
                ])
    
    return model

In [33]:
# Build our FCNN model and compile
model_fcnn = build_fcnn_model()
model_fcnn.summary()
model_fcnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS) # Compile

### Fitting Model to Training Data

In [34]:
history_fcnn = model_fcnn.fit(train_X_rgb, 
                          train_Y,  
                          epochs = 50,
                          batch_size = 128,
                          validation_split = 0.2, 
                          class_weight = classWeight, 
                          verbose = 1,
                          callbacks = [checkpoint_cb, early_stopping_cb, lr_scheduler]) # Fit the model

In [35]:
# Evaluate and display results
results = model_fcnn.evaluate(test_X_rgb, test_Y) # Evaluate the model on test data
results = dict(zip(model_fcnn.metrics_names,results))

print(results)
plottingScores(history_fcnn) # Visualise scores

## Part 3.7: Building Model #2 - CNN

In our CNN model, fewer parameters are needed because every convolutional layer reduces the dimensions of the input through the convolution operation.

In [19]:
"""
@Description: This function builds our custom CNN Model

@Inputs: N/A

@Output: Returns the CNN model
"""
def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding = 'valid', activation = 'relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding = 'valid', activation = 'relu'), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'valid'),
        tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Flatten(), # flatten output of conv
        tf.keras.layers.Dense(512, activation = "relu"), # hidden layer
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation = "relu"), #  output layer
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation = "sigmoid")])
    
    return model

In [20]:
# Build and compile model
model_cnn = build_cnn_model()
model_cnn.summary()
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [21]:
# Fit model
history_cnn = model_cnn.fit(train_X_rgb, 
                      train_Y,  
                      epochs=30, 
                      validation_split = 0.15, 
                      batch_size=128,
                      class_weight=classWeight,
                      callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
                      verbose=1) # Fit the model

In [22]:
# Evalute the models results and put into a dict
results = model_cnn.evaluate(test_X_rgb, test_Y)
results = dict(zip(model_cnn.metrics_names,results))

print(results)
plottingScores(history_cnn) # Visualise scores

## Part 3.8: Building Model #3 - Mobile Net with Transfer Learning

In [23]:
"""
@Description: This function builds our MobileNet Model

@Inputs: N/A

@Output: Returns the Mobile Net model
"""
def build_mn_model():
    
    model = tf.keras.Sequential([
        tf.keras.applications.MobileNetV2(include_top = False, weights="imagenet", input_shape=(128, 128, 3)),
        tf.keras.layers.GlobalAveragePooling2D(),
        Dense(1, activation = 'sigmoid')
    ])
    
    model.layers[0].trainable = False
    
    return model

In [24]:
# Build and compile mobile net model
model_mn = build_mn_model()
model_mn.summary()
model_mn.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

### We run our best model here on a larger portion of the training data

In [25]:
history_mn = model_mn.fit(train_X_rgb, 
                          train_Y,  
                          epochs = 30, 
                          validation_split = 0.20, 
                          class_weight = classWeight,
                          batch_size = 64,
                          callbacks = [checkpoint_cb, early_stopping_cb, lr_scheduler])

In [26]:
# Show results and print graphs
results = model_mn.evaluate(test_X_rgb, test_Y)
results = dict(zip(model_mn.metrics_names,results))

print(results)
plottingScores(history_mn) # Visualise scores

### Show Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix

y_pred = model_mn.predict_classes(test_X_rgb)
confusion_matrix(test_Y, y_pred)

### Function to Perform K-Fold CV

In [ ]:
"""
@Description: This function performs K-Fold Cross Validation with a provided Deep Learning Model

@Inputs:
    - K: Number of folds
    - build_model_func: Function to create model
    - epochs: Number of epochs to train data
    - batchSize: Batch size when fitting the model

@Output: Dict of metric results from K-fold CV
"""
def performCV(K, build_model_func, epochs, batchSize):
    
    kfold = KFold(n_splits = K, shuffle = True) # Split data into K Folds
    
    res = {
        'acc_per_fold': [],
        'precision_per_fold': [],
        'recall_per_fold': [],
        'auc_per_fold': [],
        'loss_per_fold': []
    }

    fold_no = 1

    for train_index, test_index in kfold.split(train_X_rgb):

        X_train, X_test = train_X_rgb[train_index], train_X_rgb[test_index] # Split data
        y_train, y_test = train_Y[train_index], train_Y[test_index]

        model = build_model_func() # Build model
        mets = ['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.AUC(name='AUC')]

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=mets) # Compile our model

        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Train the model on the current fold
        history = model.fit(X_train,
                            y_train, 
                            epochs = epochs,
                            batch_size = batchSize,
                            class_weight = classWeight,
                            callbacks = [checkpoint_cb, early_stopping_cb, lr_scheduler]) # Fit data to model

        scores = model.evaluate(X_test, y_test, verbose=0) # Evalute the model

        print(f'Scores for fold {fold_no}:')
        print(f'{model.metrics_names[0]}: {scores[0]}')
        print(f'{model.metrics_names[1]}: {scores[1]*100}%')
        print(f'{model.metrics_names[2]}: {scores[2]*100}%')
        print(f'{model.metrics_names[3]}: {scores[3]*100}%')

        res['loss_per_fold'].append(scores[0])
        res['acc_per_fold'].append(scores[1] * 100)
        res['precision_per_fold'].append(scores[2]*100)
        res['recall_per_fold'].append(scores[3]*100)
        res['auc_per_fold'].append(scores[4]*100)

        gc.collect()
        # Increase fold number
        fold_no += 1
    
    return res # return our results dict

## Part 3.9: K-Fold Cross Validation with all 3 Networks

In [ ]:
# Full-connected NN
resFCNN = performCV(5, build_fcnn_model, 30, 128)

In [ ]:
# Convolutional NN
resCNN = performCV(5, build_cnn_model, 30, 64)

In [ ]:
# MobileNet
resMB = performCV(5, build_mn_model, 30, 64)

In [ ]:
resMB

### Results For Architectures

In [ ]:
"""
5k Training
3k Testing

Architecture 1:
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Flatten(), # flatten output of conv
        tf.keras.layers.Dense(100, activation='relu'), # hidden layer
        tf.keras.layers.Dense(1, activation='sigmoid') #  output layer
        
    ---------------------------------------------------- Performance on Test Data ----------------------------------------------------
    { 'loss': 0.255420297, 'accuracy': 0.904666662, 'precision': 0.881006836, 'recall': 0.951792359, 'AUC': 0.968622922}
        
Architecture 2:
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Flatten(), # flatten output of conv
        tf.keras.layers.Dense(100, activation='relu'), # hidden layer
        tf.keras.layers.Dense(1, activation='sigmoid') #  output layer
    
    ---------------------------------------------------- Performance on Test Data ----------------------------------------------------
    { 'loss': 0.198399558, 'accuracy': 0.950666666, 'precision': 0.933372616, 'recall': 0.978368341, 'AUC': 0.986180067}
    
    
Architecture 3:
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        
        tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(),
        
        tf.keras.layers.Flatten(), # flatten output of conv
        tf.keras.layers.Dense(100, activation='relu'), # hidden layer
        tf.keras.layers.Dense(1, activation='sigmoid') #  output layer


    ---------------------------------------------------- Performance on Test Data ----------------------------------------------------
    {'loss': 0.1422816216, 'accuracy': 0.976333320, 'precision': 0.984952986, 'recall': 0.970951795, 'AUC': 0.991799652}

Architecture 4:
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(32, 3, activation='relu', padding='valid'),
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Flatten(), # flatten output of conv
        tf.keras.layers.Dense(100, activation='relu'), # hidden layer
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid') #  output layer
    
    
    ---------------------------------------------------- Performance on Test Data ----------------------------------------------------
    { 'loss': 0.1239979043, 'accuracy': 0.982666671, 'precision': 0.985130131, 'recall': 0.982694685, 'AUC': 0.992944598}
            
            
Architecture 5:
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(128, 128, 3)), #  convolutional layer
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        
        tf.keras.layers.Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'), #  convolutional layer
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)), # flatten output of conv
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(32, 3, activation='relu', padding='valid'),
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='valid'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.3),
        

        tf.keras.layers.Flatten(), # Flattening
        
        # Full Connection
        tf.keras.layers.Dense(64, activation='relu'), # hidden layer
        tf.keras.layers.Dropout(0.5), # Dropout
        tf.keras.layers.Dense(1, activation='sigmoid') #  output layer
        
        ---------------------------------------------------- Performance on Test Data ----------------------------------------------------
    { 'loss': 0.12123415754, 'accuracy': 0.984615671, 'precision': 0.987261581, 'recall': 0.985671211, 'AUC': 0.994511598}

"""

### Manual Hyper-parameter Tuning results for FCNN

In [ ]:
"""
Manual Hyper-parameter Tuning

batch_size=32
    cWeight: None {'loss': 0.22600425779819489, 'accuracy': 0.92166668176651, 'precision': 0.9292364716529846, 'recall': 0.9252163171768188, 'AUC': 0.9672043323516846}
    cWeight: Balanced {'loss': 0.2335905283689499, 'accuracy': 0.9136666655540466, 'precision': 0.9155963063240051, 'recall': 0.9252163171768188, 'AUC': 0.9655577540397644}
    
batch_size=64
    cWeight: None {'loss': 0.22068753838539124, 'accuracy': 0.9193333387374878, 'precision': 0.9149577617645264, 'recall': 0.9375772476196289, 'AUC': 0.9699712991714478}
    cWeight: Balanced {'loss': 0.2424456775188446, 'accuracy': 0.9079999923706055, 'precision': 0.8829908967018127, 'recall': 0.956118643283844, 'AUC': 0.9677296280860901}

batch_size=128
    cWeight: None {'loss': 0.23750829696655273, 'accuracy': 0.9100000262260437, 'precision': 0.8855835199356079, 'recall': 0.95673668384552, 'AUC': 0.9694961309432983}
    cWeight: Balanced {'loss': 0.2239508330821991, 'accuracy': 0.9196666479110718, 'precision': 0.9100655317306519, 'recall': 0.94437575340271, 'AUC': 0.9697944521903992}

batch_size=256
    cWeight: None {'loss': 0.2305724024772644, 'accuracy': 0.9190000295639038, 'precision': 0.9109384417533875, 'recall': 0.9419035911560059, 'AUC': 0.9681356549263}
    cWeight: Balanced {'loss': 0.22952377796173096, 'accuracy': 0.9203333258628845, 'precision': 0.9171203970909119, 'recall': 0.9369592070579529, 'AUC': 0.9694135785102844}

"""